# Imports

In [ ]:
import tensorflow as tf
from keras import Model
import matplotlib.pyplot as plt
import cv2
from keras.applications import DenseNet121
from keras.layers import *
from keras.callbacks import *
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

# GPU

In [ ]:
!nvidia-smi

# Creating variables containing frequently used directories, and printing out number of images in data set

In [ ]:
for dirpath, dirnames, filenames in os.walk("C:/Users/Jakub/PycharmProjects/BigDataProjekt/data_classes"):
    if len(filenames) != 0:
        print(f"{len(filenames)} images in '{dirpath}'.")

train_dir = "C:/Users/Jakub/PycharmProjects/BigDataProjekt/data_classes/train/"
test_dir = "C:/Users/Jakub/PycharmProjects/BigDataProjekt/data_classes/test/"
val_dir = "C:/Users/Jakub/PycharmProjects/BigDataProjekt/data_classes/val/"

classes = []
for name in os.listdir(train_dir):
    classes.append(name)

print(classes)

img_array = cv2.imread("C:/Users/Jakub/PycharmProjects/BigDataProjekt/data_classes/train/Glioma/0.png", 1)
plt.imshow(np.real(img_array))
plt.title("Glioma sample")
plt.show()

# Converting dataset

In [ ]:
# Declaring ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  rotation_range=5)
test_datagen = ImageDataGenerator(rescale = 1/255)

In [ ]:
IMAGE_SHAPE = (256, 256)
BATCH_SIZE = 32

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")

val_data = train_datagen.flow_from_directory(val_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")

# Initializing model

In [ ]:
base_model = DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=Input(shape=(IMAGE_SHAPE+(3,)))
)

In [ ]:
# Adding weight classes

from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_data.classes),
                                                  y=train_data.classes)
class_weights = dict(enumerate(class_weights))

In [ ]:
# Addding head of the model

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256, activation='relu')(x)
x=Dropout(0.25)(x)
prediction=Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs = prediction)

model.summary()

In [ ]:
# Compiling model

model.compile(optimizer=tf.keras.optimizers.Adam(.5e-5),
    loss="BinaryCrossentropy",
    metrics=["accuracy"],
    )

In [ ]:
# Initializing checkpoint class

train_steps = train_data.samples // BATCH_SIZE
test_steps= test_data.samples // BATCH_SIZE
val_steps = val_data.samples // BATCH_SIZE

callback = ModelCheckpoint("best_model.hdf5",
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=True,
                           mode='auto',
                           save_freq="epoch")

print(train_steps,  test_steps, val_steps)

In [ ]:
# Fitting existing model with freezed trainable layers to our data

history = model.fit(
    train_data,
    steps_per_epoch=train_steps,
    epochs = 20,
    validation_data=val_data,
    validation_steps=val_steps,
    callbacks=[callback],
    class_weight=class_weights
)

# Accuracy with best weights

In [ ]:
model.load_weights("best_model.hdf5") # Loading best weights that occured during all epochs

print("Accuracy:", round(model.evaluate(test_data, steps=test_steps, return_dict=True)["accuracy"], 4)*100, "%")

# Plotting model's preformance

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(len(history.history['loss']))

plt.plot(epochs, loss, label='training_loss')
plt.plot(epochs, val_loss, label='val_loss')
plt.axvline(x=np.argmin(history.history["val_loss"]), color="green", label='last epoch with best val_loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.xticks(ticks=np.arange(0, len(history.history['loss'])), rotation = 60, fontsize=6)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.figure()
plt.plot(epochs, accuracy, label='training_accuracy')
plt.plot(epochs, val_accuracy, label='val_accuracy')
plt.axvline(x=np.argmin(history.history["val_loss"]), color="green", label='last epoch with best val_loss')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.xticks(ticks=np.arange(0, len(history.history['loss'])), rotation = 60, fontsize=6)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Confusion matrix

In [ ]:
y_pred_probs = model.predict_generator(test_data)

y_pred_labels = np.argmax(y_pred_probs, axis=1)

y_true_labels = test_data.classes

cm = confusion_matrix(y_true_labels, y_pred_labels)

# Get the class labels from the generator
class_labels = list(test_data.class_indices.keys())

# Create a heatmap of the confusion matrix
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=class_labels, yticklabels=class_labels)

# Set the axis labels
plt.xlabel('Predicted')
plt.ylabel('True')

# Set the title
plt.title('Confusion Matrix')

# Show the plot
plt.show()